In [1]:
import json, sqlite3, urllib.request
import pandas as pd
from datetime import datetime

In [2]:
# open database
conn = sqlite3.connect('log.db')
c = conn.cursor()

In [3]:
# create table
c.execute("CREATE TABLE IF NOT EXISTS log_802 (id INTEGER PRIMARY KEY AUTOINCREMENT, \
    timestamp DATETIME NOT NULL, vehicle_id INTEGER NOT NULL, route_id INTEGER NOT NULL, \
    lat REAL NOT NULL, lon REAL NOT NULL, direction REAL NOT NULL)")

In [4]:
# get vehicles data from API
req = urllib.request.Request('http://api.metro.net/agencies/lametro-rail/routes/801/vehicles/')
req.add_header('Accept', 'application/json')
response = urllib.request.urlopen(req)
data = response.read()

In [5]:
# quick peek
encoding = response.info().get_content_charset('utf-8')
json.loads(data.decode(encoding))

{'items': [{'heading': 270.0,
   'id': '107',
   'latitude': 33.928426,
   'longitude': -118.230135,
   'predictable': True,
   'route_id': '801',
   'run_id': '801_1_var0',
   'seconds_since_report': 12},
  {'heading': 0.0,
   'id': '101',
   'latitude': 33.9753,
   'longitude': -118.243243,
   'predictable': True,
   'route_id': '801',
   'run_id': '801_0_var0',
   'seconds_since_report': 31},
  {'heading': 180.0,
   'id': '111',
   'latitude': 34.031664,
   'longitude': -118.266014,
   'predictable': True,
   'route_id': '801',
   'run_id': '801_1_var0',
   'seconds_since_report': 19},
  {'heading': 180.0,
   'id': '103',
   'latitude': 33.788987,
   'longitude': -118.189398,
   'predictable': True,
   'route_id': '801',
   'run_id': '801_1_var0',
   'seconds_since_report': 19},
  {'heading': 180.0,
   'id': '110',
   'latitude': 33.9234409,
   'longitude': -118.235169,
   'predictable': True,
   'route_id': '801',
   'run_id': '801_1_var0',
   'seconds_since_report': 10},
  {'headi

In [6]:
# derive timestamp from datetime string in returned header
time_string = response.info()['Date']
date_obj = datetime.strptime(time_string, '%a, %d %b %Y %H:%M:%S %Z')
ts = date_obj.timestamp()

In [7]:
# load data into dict object and access array of items
parsed = json.loads(data)
items = parsed["items"]

In [8]:
# these fields must be present in each item for data to be valid
fields = ['id', 'seconds_since_report', 'route_id', 'latitude', 'longitude', 'heading']
# list of valid items
valid = []
# check all fields are present
for vehicle in items:
    for field in fields:
        if not field in vehicle.keys():
            all_fields_present = False
            break
        else:
            all_fields_present = True
    if all_fields_present == True:
        # push vehicle into valid array
        valid.append(vehicle)

for vehicle in valid:
    # calculate timestamp of each position entry and add key to dict
    vehicle['timestamp'] = ts - int(vehicle['seconds_since_report'])
    # insert each vehicle position entry into database
    c.execute("INSERT INTO log_802 (timestamp, vehicle_id, route_id, lat, lon, direction) VALUES (?, ?, ?, ?, ?, ?)", [float(vehicle['timestamp']), int(vehicle['id']),int(vehicle['route_id']), float(vehicle['latitude']), float(vehicle['longitude']), float(vehicle['heading'])])

conn.commit()

# close database
conn.close()
print("DONE")

DONE


In [9]:
# insert the database into a dataframe
new_conn = sqlite3.connect('log.db')
df = pd.read_sql_query("SELECT * from log_802", new_conn)

In [10]:
df.head()

,id,timestamp,vehicle_id,route_id,lat,lon,direction
0,1,1531483316,104,801,33.790072,-118.189292,180.0
1,2,1531483307,107,801,33.872398,-118.221081,0.0
2,3,1531483307,101,801,33.794189,-118.189324,0.0
3,4,1531483316,111,801,34.033758,-118.270421,0.0
4,5,1531483278,103,801,33.959738,-118.243314,180.0


In [11]:
# enjoy